This notebook serves as a reproduction and introduction to the following scholarly article:   

@article{hansen2021frontiers,  
  title={**Frontiers: Algorithmic collusion: Supra-competitive prices via independent algorithms**},  
  author={Hansen, Karsten T and Misra, Kanishka and Pai, Mallesh M},  
  journal={**Marketing Science**},  
  volume={40},  
  number={1},  
  pages={1--12},  
  year={2021},  
  publisher={INFORMS}  
}

**TL, DR**:

This paper analysis the outcomes when two competing sllers (duopoly markets) use UCB algorithm to price homogeneious product dynamically. 

The profit function for each firm is stochastic: $\pi_i(p_i, p_{-i}, \delta)$, where $\delta$ is clled the signal-to-noise ration (**SNR**). A higher $\delta$ indicates lower stochasticity in profit.

The system dynamics yield two possible outcomes:

1. Convergence to Nash equilibrum;

2. Convergence to a **supra-competitive** scenario resembling monopoly pricing.

The learning algorithm is misspecified,  as the UCB algorithm neglects consideration of the competitor's decision.

However, the numerical experiments reveal that the outcome depends on the SNR:

1. when SNR is low: end up with the Nash-equilibrum;

2. when SNR is high: end up with the supra-competitive scenario.

The core of this paper consists of numerical expirements, complemented by straightforward mathematical proofs in the appendix.

In [1]:
import numpy as np
np.random.seed(93)
import seaborn as sns
import matplotlib.pyplot as plt

# Part 1: The Model Setting


Assume there are 2 firms (denoted by $i, j$) selling a homogeneous product within an infinte time period. At each period $t$, firm $i$ gives price $p_{i,t}$ from a discrete space $P=\{p_1, p_2, \cdots, p_K\}$. After facing the price $p_{-i,t}$ by the competitor, the profit can be obtained through a parametric data-generating process (DGP):

$$
\pi_{i,t} = (\alpha - \beta p_{i,t} + \gamma p{-i, t})\cdot p_{i,t} + \epsilon_t, \\
\epsilon_t = \sim U[-\frac1\delta,\frac1\delta],
$$
where $\delta$ is called the signal-to-noise ration (**SNR**).

From the DGP we can observe:

1. The marginal cost of the product is zero;
2. The demand function is deterministic, the random term is added to the profit only;
3. The SNR is inversely proportional to the stability of the profit;
4. The demand function is symmetric for both firms.

Under such settings, we can calculate the prices under Nash equilibrium (the competitive case) or under collusive pricing (the supra-competitive case) analytically:

$$
Collusive: p^D = \frac{\alpha}{2\beta-\gamma} \\
Collusive: p^M = \frac{\alpha}{2(\beta-\gamma)}
$$

In [2]:
class Hansen_env():

    def __init__(self, info):
        """
        The input parameters info should be dictionary.
        info should contain keys: 'alpha', 'beta', 'gamma', 'delta'
        """
        self.info = info
        self.rewards_log = []
        self.actions_log = []
        self.prices_log = []
        self.action_spaces = None
        self.nash = None
        self.collusion = None
        self.check()
        self.initialize()
    
    def check(self):
        keys = ['alpha', 'beta', 'gamma', 'delta']
        if not all(key in self.info for key in keys):
            raise ValueError("Incorrect Parameters")
        
    def step(self, actions):
        self.actions_log.append(actions)
        prices = self.action_spaces[0][actions[0]], self.action_spaces[1][actions[1]]
        self.prices_log.append(prices)
        rewards = self.execute(prices)
        self.rewards_log.append(rewards)
        return rewards
    
    def execute(self, prices):
        profit1 = (prices[0] * (self.info['alpha']-self.info['beta']*prices[0]+self.info['gamma']*prices[1])) + np.random.uniform(-1/self.info['delta'], 1/self.info['delta'])
        profit2 = (prices[1] * (self.info['alpha']-self.info['beta']*prices[1]+self.info['gamma']*prices[0])) + np.random.uniform(-1/self.info['delta'], 1/self.info['delta'])
        return (profit1, profit2)
    
    def initialize(self, lower = 1, upper = 2.5, space = 0.01):
        """
        Compute the nash, collusion prices and the actions spaces.
        """
        nash = self.info['alpha'] / (2*self.info['beta']-self.info['gamma']) 
        self.nash = np.array([nash, nash])
        collusion = 0.5 * self.info['alpha'] / (self.info['beta']-self.info['gamma']) 
        self.collusion = np.array([collusion, collusion])
        self.action_spaces = [np.array([nash, collusion]), np.array([nash, collusion])]

In the baseline model, the action space for each agent is $2$, meaning each agent can only pick price from the $p^D$ and $p^M$. 

The agent utilizes $UCB-tuned$ algorithm to make decision at time $t$:

\begin{aligned}
V_{k,t}& =\overline{\pi_{k,t}^{2}}-\bar{\pi}_{k,t}^{2}+\sqrt{\frac{2\log t}{n_{k,t}}},  \\
\mathrm{UCB-tuned}_{k,t}& =\bar{\pi}_{k,t}+\sqrt{\frac{\log t}{n_{k,t}}\mathrm{min}\left(\frac14,\mathrm{V}_{k,t}\right),} 
\end{aligned}

where \(n_{k, t}\) represents the frequency of the agent taking action \(k\) up to time \(t\).To give an intuitive understanding on the formula:

1. The term $\overline{\pi_{k,t}^{2}}-\bar{\pi}_{k,t}^{2}$ is the empirical variance.

2. The term $\sqrt{\frac{\log t}{n_{k,t}}\mathrm{min}(\frac14,\mathrm{V}_{k,t})}$ is used for exploratin.

The agent pick action $k$ at time $t$ with the highest index $\mathrm{UCB-tuned}_{k,t}$.


In [3]:
class Hansen_agent:
    def __init__(self):
        self.t = 0
        self.n = np.array([0, 0])
        self.last_action = None
        self.reward = [[], []]

    def action(self):
        pi = np.array((np.mean(self.reward[0]), np.mean(self.reward[1])))
        Pi = np.array((np.mean(np.array(self.reward[0]) ** 2), np.mean(np.array(self.reward[1]) ** 2)))
        temp = np.sqrt(2 * np.log(self.t) / self.n)
        V = Pi - pi**2 + temp
        temp = np.log(self.t) / self.n
        temp *= np.minimum(0.25, V)
        UCB = pi + np.sqrt(temp)
        action = np.argmax(UCB)
        self.last_action = action
        return action

    def update(self, reward):
        self.t += 1
        self.n[self.last_action] += 1
        self.reward[self.last_action].append(reward)       

Because the agents have no information on the bandit at the beginning of the game, they explore randomly at the first two rounds:

1. In the first round, each agent randomly selects one action.
2. In the second round, each agent chooses the opposite action to that of the first round (there are 2 actins in total).
3. Subsequently, each agent makes decisions using the UCB-tuned algorithm.

In [4]:
def initialize(env, agent1, agent2):
    action1 = np.random.choice(np.array(2), 2)
    agent1.last_action, agent2.last_action = action1
    rewards = env.step((agent1.last_action, agent2.last_action))
    agent1.update(rewards[0])
    agent2.update(rewards[1])
    agent1.last_action, agent2.last_action = 1 - action1
    rewards = env.step((agent1.last_action, agent2.last_action))
    agent1.update(rewards[0])
    agent2.update(rewards[1])

Now, we can use the parameters set in the paper to construct an environment for interaction.

In [5]:
info = {'alpha': 0.48, 'beta': 0.9, 'gamma': 0.6, 'delta': 0.1}
env = Hansen_env(info)
agent1, agent2 = Hansen_agent(), Hansen_agent()
initialize(env, agent1, agent2)


# Part 2: The Main Resuts

In the paper, the authors dicuss the impact of *SNR* on the pricing outcome. 

For each given SNR, 500 Monte Carlo Simulations are performed. For each simulation, we consider the **median price** charged in the last 1,000 rounds out of two million rounds.

In [6]:
delta_list = [0.1, 0.2, 0.4, 1, 2.5, 5, 10]
results = {delta:[] for delta in delta_list}
for delta in delta_list:
    print(delta)
    for _ in range(500):
        info = {'alpha': 0.48, 'beta': 0.9, 'gamma': 0.6, 'delta': 0.1}
        env = Hansen_env(info)
        agent1, agent2 = Hansen_agent(), Hansen_agent()
        initialize(env, agent1, agent2)
        for __ in range(2000000 - 2):
            actions = (agent1.action(), agent2.action())
            rewards = env.step(actions)
            agent1.update(rewards[0])
            agent2.update(rewards[1])
        prices = np.array(env.prices_log)[-1000:, :]
        price1, price2 = np.median(prices[:, 0]), np.median(prices[:, 1])
        results[delta].append([price1, price2])

0.1


KeyboardInterrupt: 

In [14]:
for __ in range(100):
    actions = (agent1.action(), agent2.action())
    rewards = env.step(actions)
    agent1.update(rewards[0])
    agent2.update(rewards[1])

In [15]:
prices = np.array(env.prices_log)[-1000:, :]
price1, price2 = np.median(prices[:, 0]), np.median(prices[:, 1])
results[delta].append([price1, price2])

In [17]:
results[delta]

[[0.7999999999999998, 0.3999999999999999]]

In [16]:
data = results[delta]
sns.kdeplot(x=data[:, 0], y=data[:, 1], fill=True, cmap="Blues", cbar=True)
plt.title("2D Density Plot")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()

TypeError: list indices must be integers or slices, not tuple

# Part 3: Robust Tests & Proof